# Gathering and cleaning data:

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

## Source of data
This project uses publicly available [datasets](https://grouplens.org/datasets/movielens/) from [MovieLens](https://www.movielens.org).

## Read-in Data:

In [2]:
!pwd

/Users/OwenSwetenburg/git_personal/capstone_recommender_system


In [3]:
movies = pd.read_csv('datasets/ml-latest-small/movies.csv')

ratings = pd.read_csv('datasets/ml-latest-small/ratings.csv')

tags = pd.read_csv('datasets/ml-latest-small/tags.csv')

links = pd.read_csv('datasets/ml-latest-small/links.csv')

## Initial EDA:

### Movies:
- the year should remain in the title column in order to differentiate remakes (i.e. Ghostbusters 
- genres should be split on the pipe ("|") and dummified
    - according to the [MovieLens documentation](http://files.grouplens.org/datasets/movielens/ml-latest-README.html) there are 20 genres

In [4]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
movies_film_counts = len(movies['movieId'].unique())
print(f'There are {movies_film_counts:,} films in the movies dataframe.')

movies_nulls = movies.isnull().sum().sum()
print(f'There are {movies_nulls} null values in the movies dataframe.')

There are 9,742 films in the movies dataframe.
There are 0 null values in the movies dataframe.


In [6]:
movies.dtypes

movieId     int64
title      object
genres     object
dtype: object

### Ratings:

In [7]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [8]:
ratings_rating_counts = len(ratings['rating'])
print(f'There are {ratings_rating_counts:,} ratings in the ratings dataframe.')

ratings_user_counts = len(ratings['userId'].unique())
print(f'There are {ratings_user_counts:,} users who rated at least one film in the ratings dataframe.')

ratings_nulls = ratings.isnull().sum().sum()
print(f'There are {ratings_nulls} null values in the ratings dataframe.')

There are 100,836 ratings in the ratings dataframe.
There are 610 users who rated at least one film in the ratings dataframe.
There are 0 null values in the ratings dataframe.


In [9]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

### Tags:

In [10]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [11]:
tags_tag_counts = tags.shape[0]
print(f'There are {tags_tag_counts:,} unique tags in the tags dataframe.')

tags_film_counts = len(tags['tag'].unique())
print(f'There are {tags_film_counts:,} films that have been tagged in the tags dataframe.')

tags_user_counts = len(tags['userId'].unique())
print(f'There are {tags_user_counts:,} users that have applied at least one tag in the tags dataframe.')

tags_nulls = tags.isnull().sum().sum()
print(f'There are {tags_nulls:,} null values in the tags dataframe.')

There are 3,683 unique tags in the tags dataframe.
There are 1,589 films that have been tagged in the tags dataframe.
There are 58 users that have applied at least one tag in the tags dataframe.
There are 0 null values in the tags dataframe.


In [12]:
tags.dtypes

userId        int64
movieId       int64
tag          object
timestamp     int64
dtype: object

### Links:

'movieId' is the movielens id <br>
'imdbId' is the Internet Movie Database id <br>
'tmdbId' id is The Movie DB id

In [13]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [14]:
links_film_counts = links.shape[0]
print(f'There are {links_film_counts:,} unique films with links in the links dataframe.')

links_nulls = links.isnull().sum().sum()
print(f'There are {links_nulls:,} null values in the links dataframe.')

There are 9,742 unique films with links in the links dataframe.
There are 8 null values in the links dataframe.


There are 8 missing values from the tmdbId column

In [15]:
links.isnull().sum()

movieId    0
imdbId     0
tmdbId     8
dtype: int64

In [16]:
links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

## Merge Data Frames

### Merge Movies and Links

In [17]:
movies.shape

(9742, 3)

In [18]:
links.shape

(9742, 3)

In [19]:
df = pd.merge(movies, links, how='inner', on='movieId')
df.head(3)

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0


## Addding features to the dataframe

### Extract year from title

In [20]:
def get_years(title):
    title_slice = title.strip()[-5:-1]
    if title_slice.isdigit():
        return int(title_slice)
    else:
        return 0

In [21]:
df['year'] = df['title'].map(get_years)

In [22]:
df['year'].value_counts

<bound method IndexOpsMixin.value_counts of 0       1995
1       1995
2       1995
3       1995
4       1995
        ... 
9737    2017
9738    2017
9739    2017
9740    2018
9741    1991
Name: year, Length: 9742, dtype: int64>

In [23]:
df['year'].sort_values(ascending=True)

9259       0
9179       0
9525       0
9515       0
9514       0
        ... 
9708    2018
9709    2018
9710    2018
9712    2018
9700    2018
Name: year, Length: 9742, dtype: int64

In [24]:
# def get_years(titles):
#     for title in titles:
#         title_slice = title[-5:-1]
#         if title_slice.isdigit():
#             return df.title.apply(lambda x: x[-5:-1])
#         else:
#             return 0

#### Manually imputing missing years

In [25]:
df[df['year'] == 0]

,movieId,title,genres,imdbId,tmdbId,year
6059,40697,Babylon 5,Sci-Fi,105946,NaN,0
9031,140956,Ready Player One,Action|Sci-Fi|Thriller,1677720,333339.0,0
9091,143410,Hyena Road,(no genres listed),4034452,316042.0,0
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),229922,127605.0,0
9179,149334,Nocturnal Animals,Drama|Thriller,4550098,340666.0,0
9259,156605,Paterson,(no genres listed),5247022,370755.0,0
9367,162414,Moonlight,Drama,4975722,376867.0,0
9448,167570,The OA,(no genres listed),4635282,432192.0,0
9514,171495,Cosmos,(no genres listed),81846,409926.0,0
9515,171631,Maria Bamford: Old Baby,(no genres listed),6264596,455601.0,0


The four television shows need to be removed <br>(Babylon 5, The Adventures of Sherlock Holmes and Doctor Watson, The OA, and Cosmos)

In [26]:
df = df.drop([6059, 9138, 9448, 9611])

Manually imputing the years of the following eight films

In [27]:
df.loc[9031, 'year'] = 2018
df.loc[9091, 'year'] = 2015
df.loc[9179, 'year'] = 2016
df.loc[9259, 'year'] = 2016
df.loc[9367, 'year'] = 2016
df.loc[9514, 'year'] = 1980
df.loc[9515, 'year'] = 2017
df.loc[9525, 'year'] = 2017

In [28]:
df[df['year'] == 0]

,movieId,title,genres,imdbId,tmdbId,year


In [29]:
df['year'].sort_values(ascending=True)

5868    1902
6355    1903
9020    1908
4743    1915
6863    1916
        ... 
9709    2018
9710    2018
9681    2018
9700    2018
9699    2018
Name: year, Length: 9738, dtype: int64

### Getting the Decade

In [30]:
def get_decade(year):
    try:
        return str(int(str(year)[:-1]) * 10) + 's'
    except:
        return 0

In [31]:
df['decade'] = df['year'].map(get_decade)
df.head(3)

,movieId,title,genres,imdbId,tmdbId,year,decade
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990s
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,1995,1990s
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995,1990s


In [32]:
df['decade'].sort_values()

5868    1900s
9020    1900s
6355    1900s
6863    1910s
5217    1910s
        ...  
8215    2010s
8216    2010s
8217    2010s
8210    2010s
7959    2010s
Name: decade, Length: 9738, dtype: object

### Count vectorize genres column

In [33]:
def genre_tokenizer(genres):
    return genres.split('|')

In [34]:
df['genres'][:5].map(genre_tokenizer)

0    [Adventure, Animation, Children, Comedy, Fantasy]
1                       [Adventure, Children, Fantasy]
2                                    [Comedy, Romance]
3                             [Comedy, Drama, Romance]
4                                             [Comedy]
Name: genres, dtype: object

In [35]:
cv = CountVectorizer(tokenizer=genre_tokenizer)

In [36]:
cv.fit(df['genres'])
vectorized_genres = cv.transform(df['genres'])

In [37]:
genres_vectorized = pd.DataFrame(vectorized_genres.todense(), columns=cv.get_feature_names())
genres_vectorized.head(3)

,(no genres listed),action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


## Merge genre_vectorized with df

In [38]:
genres_vectorized.shape

(9738, 20)

In [39]:
df.shape

(9738, 7)

In [40]:
# df = pd.merge(movies, links, how='inner', on='movieId')
# df.head(3)

In [41]:
df = pd.merge(df, genres_vectorized, how='inner', left_index=True, right_index=True)
df.head(3)

,movieId,title,genres,imdbId,tmdbId,year,decade,(no genres listed),action,adventure,...,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990s,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,1995,1990s,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995,1990s,0,0,0,...,0,0,0,0,0,1,0,0,0,0


## Save the datframe to a csv for EDA

In [42]:
df.to_csv('datasets/movie_dataframe_no_ratings.csv')

## Merge Movies, Links, and Ratings

Time to finally merge the ratings dataframe with the movies and links dataframe.

In [43]:
ratings.shape

(100836, 4)

In [44]:
df_with_ratings = pd.merge(df, ratings, how='inner', on='movieId')
df_with_ratings.shape

(100827, 30)

In [45]:
df_with_ratings.head(3)

,movieId,title,genres,imdbId,tmdbId,year,decade,(no genres listed),action,adventure,...,musical,mystery,romance,sci-fi,thriller,war,western,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990s,0,0,1,...,0,0,0,0,0,0,0,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990s,0,0,1,...,0,0,0,0,0,0,0,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995,1990s,0,0,1,...,0,0,0,0,0,0,0,7,4.5,1106635946


In [46]:
df_with_ratings.to_csv('datasets/dataframe_for_modeling.csv')